Кустарная тетрадка с наглядным примером по запуску и тестированию стратегий

ссылка{https://kernc.github.io/backtesting.py/} на документацию

ссылка{https://youtu.be/e4ytbIm2Xg0} на видеоурок по библиотечке

ссылка{https://youtu.be/xljQpeYQYkI} на туториал по использованию кастомных индикаторов

In [1]:
# !pip install backtesting
# !pip install python_binance
# !pip install pandas


## Выгрузка данных

In [2]:
from binance import Client
import pandas as pd
import pandas_ta as pta


In [3]:
interval = "15m"
symbol = "ETHUSDT"
end_date = "96 hours ago UTC"


In [4]:
def get_actual_data(
    symbol: str,
    time_frame: str = "1m",
    end_date: str = "10 hours ago UTC",
) -> pd.DataFrame:
    client = Client()
    df = pd.DataFrame(client.get_historical_klines(symbol, time_frame, end_date))

    df = df.iloc[:, 0:6]
    df.columns = ["Time", "Open", "High", "Low", "Close", "Volume"]
    df.set_index("Time", inplace=True)
    df.index = pd.to_datetime(df.index, unit="ms")
    df = df.astype(float)
    return df


In [5]:
data = get_actual_data(symbol, time_frame=interval, end_date=end_date)


In [6]:
data


,Open,High,Low,Close,Volume
Time,,,,,
2024-01-17 15:15:00,2561.35,2562.04,2548.27,2550.85,7912.7020
2024-01-17 15:30:00,2550.84,2554.59,2541.83,2544.51,6830.8334
2024-01-17 15:45:00,2544.51,2548.93,2536.40,2539.46,7118.9596
2024-01-17 16:00:00,2539.47,2542.91,2537.33,2541.41,4973.4638
2024-01-17 16:15:00,2541.42,2544.75,2533.79,2537.39,5339.1087
...,...,...,...,...,...
2024-01-21 14:00:00,2479.36,2481.05,2478.80,2480.20,1366.1303
2024-01-21 14:15:00,2480.19,2481.03,2475.00,2476.15,2815.4651
2024-01-21 14:30:00,2476.15,2479.05,2475.57,2477.75,3428.9375


## Стратегия

In [7]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import SMA


In [8]:
def _get_ema(close, index, length):
    df = pd.DataFrame({"data": list(index), "close": list(close)}).set_index("data")
    return list(pta.ema(df.close, length=length))

def _get_rsi(close, index, length):
    df = pd.DataFrame(({"data": list(index), "close": list(close)})).set_index("data")
    return list(pta.rsi(df.close, length=length))

def _get_macd_main_line(close, index, fast, slow, signal):
    df = pd.DataFrame({"data": list(index), "close": list(close)}).set_index("data")
    return list(pta.macd(df.close, fast=fast, slow=slow, signal=signal)["MACD_12_26_9"])

def _get_macd_signal_line(close, index, fast, slow, signal):
    df = pd.DataFrame({"data": list(index), "close": list(close)}).set_index("data")
    return list(pta.macd(df.close, fast=fast, slow=slow, signal=signal)["MACDs_12_26_9"])

def _get_macd_hist(close, index, fast, slow, signal):
    df = pd.DataFrame({"data": list(index), "close": list(close)}).set_index("data")
    return list(pta.macd(df.close, fast=fast, slow=slow, signal=signal)["MACDh_12_26_9"])


In [9]:
data["EMA_6"] = _get_ema(data.Close, data.index, length=6)
data["EMA_22"] = _get_ema(data.Close, data.index, length=22)
data["EMA_127"] = _get_ema(data.Close, data.index, length=127)


In [10]:
data[["EMA_6", "EMA_22", "EMA_127"]].describe()


,EMA_6,EMA_22,EMA_127
count,379.000000,363.000000,258.000000
mean,2487.274791,2486.508883,2483.775766
std,26.662746,24.713303,10.402665
min,2438.857164,2460.061249,2472.710028
25%,2468.263504,2468.370993,2473.901219
50%,2477.290181,2475.832265,2483.397544
75%,2512.935760,2490.706776,2489.636285
max,2542.765917,2537.363322,2516.262205


In [ ]:
self.green = self.I(_get_ema, self.data.Close, self.data.index, self.ema_period_green, name="ema_green", color=colors_5[0])
self.yellow = self.I(_get_ema, self.data.Close, self.data.index, self.ema_period_yellow, name="ema_yellow", color=colors_5[1])
self.red = self.I(_get_ema, self.data.Close, self.data.index, self.ema_period_red, name="ema_red", color=colors_5[2])

# Индикаторы для 15
self.green_15_min = _get_ema(self.data.Close, self.data.index, self.ema_period_green, step=3)
self.yellow_15_min = _get_ema(self.data.Close, self.data.index, self.ema_period_yellow, step=3)
self.red_15_min = _get_ema(self.data.Close, self.data.index, self.ema_period_red, step=3)

# Индикаторы для 30
self.green_30_min = _get_ema(self.data.Close, self.data.index, self.ema_period_green, step=6)
self.yellow_30_min = _get_ema(self.data.Close, self.data.index, self.ema_period_yellow, step=6)
self.red_30_min = _get_ema(self.data.Close, self.data.index, self.ema_period_red, step=6)

# Индикаторы для 60
self.green_60_min = _get_ema(self.data.Close, self.data.index, self.ema_period_green, step=12)
self.yellow_60_min = _get_ema(self.data.Close, self.data.index, self.ema_period_yellow, step=12)
self.red_60_min = _get_ema(self.data.Close, self.data.index, self.ema_period_red, step=12)

# Индикаторы для 240
self.green_240_min = _get_ema(self.data.Close, self.data.index, self.ema_period_green, step=48)
self.yellow_240_min = _get_ema(self.data.Close, self.data.index, self.ema_period_yellow, step=48)
self.red_240_min = _get_ema(self.data.Close, self.data.index, self.ema_period_red, step=48)

In [ ]:
def should_short(self, red, yellow, green,
                     prev_red, prev_yellow, prev_green,
                     high, prev_high, close_price, prev_close, current_volume,
                     rsi_current, upper_band, bearish_divergence):
        overbought_rsi =  (rsi_current >= 30)
        trend_down = lambda r, y, g: r[-1] >= y[-1] + self.depth_params_2 and y[-1] >= g[-1] + self.depth_params_2
        return self.check_trend_conditions(trend_down) and \
               overbought_rsi and \
               not bearish_divergence
        # return red >= yellow + self.depth_params_2 and yellow >= green + self.depth_params_2 and \
        #         overbought_rsi and not bearish_divergence

def should_long(self, red, yellow, green,
                prev_red, prev_yellow, prev_green,
                low, prev_low, close_price, prev_close, current_volume,
                rsi_current, lower_band, bullish_divergence):
    oversold_rsi = rsi_current > self.oversold_rsi_value
    trend_up = lambda r, y, g: r[-1] <= y[-1] - self.depth_params_2 and y[-1] <= g[-1] - self.depth_params_2
    return self.check_trend_conditions(trend_up) and \
            oversold_rsi and \
            not bullish_divergence

In [34]:
class TrafficLight2(Strategy):
    def init(self):
        self.close = self.data.Close
        self.high = self.data.High
        self.low = self.data.Low
        self.depth_params_1 = 0
        self.depth_params_2 = 0
        self.green = self.I(_get_ema, self.data.Close, self.data.index, 6,name="EMA_6", color="green")
        self.yellow = self.I(_get_ema, self.data.Close, self.data.index, 22, name="EMA_22", color="yellow")
        # self.red = self.I(_get_ema, self.data.Close, self.data.index, 127, name="EMA_127", color="red")
        self.red = _get_ema(self.data.Close, self.data.index, 127)
        self.not_first_short = False
        self.not_first_long = False

    def next(self):
        current_time = self.data.index[-1]
        close_price = self.data.Close[-1]
        open_price = self.data.Open[-1]
        previous_close = self.data.Close[-2]
        previous_high = self.data.High[-2]
        previous_low = self.data.Low[-2]
        previous_red = self.red[-2]
        previous_yellow = self.yellow[-2]
        previous_green = self.green[-2]
        high = self.data.High[-1]
        low = self.data.Low[-1]
        red = self.red[-1]
        yellow = self.yellow[-1]
        green = self.green[-1]
        was_hit = False

        if red >= yellow + self.depth_params_2 and yellow >= green + self.depth_params_2 and not self.position.is_short:
            self.position.close()
            self.sell(size=0.15, sl=close_price*1.005, tp=close_price*0.99)
            self.not_first_short = True
            self.not_first_long = False

        elif red <= yellow - self.depth_params_2 and yellow <= green - self.depth_params_2 and not self.position.is_long:
            self.position.close()
            self.buy(size=0.15, sl=close_price * 0.995, tp=close_price * 1.015)
            self.not_first_long = True
            self.not_first_short = False

        for trade in self.trades:
            trade_open_price = trade.entry_price
            if close_price - trade_open_price > .003 * trade_open_price and trade.is_long:
                trade.sl = trade_open_price
                trade.tp = 1.02 * trade_price
            if trade_open_price - close_price  > .003 * trade_open_price and trade.is_short:
                trade.sl = trade_open_price
                trade.tp = 0.98 * trade_price
            # if current_time - trade.entry_time > pd.Timedelta(minutes=45):

            #     # trade.exp_time = current_time + pd.Timedelta(minutes=45)
            #     # if trade.is_long:
            #     #     trade.sl = max(trade.sl, low)
            #     # else:
            #     #     trade.sl = min(trade.sl, high)
            #     self.position.close()

        # for trade in self.trades:


## Бектест

In [35]:
from backtesting import Backtest


In [36]:
cash = 100000
commission = .0004
bt_traffic_light = Backtest(data, TrafficLight2, cash=cash, commission=commission, margin=0.2)
output = bt_traffic_light.run()


<Trade size=29 time=23- price=2540.895952- pl=-60>
<Trade size=29 time=23- price=2540.895952- pl=-61>
<Trade size=29 time=23- price=2540.895952- pl=-228>
<Trade size=29 time=49- price=2529.731488- pl=24>
<Trade size=29 time=49- price=2529.731488- pl=6>
<Trade size=29 time=49- price=2529.731488- pl=-13>
<Trade size=29 time=49- price=2529.731488- pl=66>
<Trade size=29 time=49- price=2529.731488- pl=67>
<Trade size=29 time=49- price=2529.731488- pl=4>
<Trade size=29 time=49- price=2529.731488- pl=-58>
<Trade size=29 time=49- price=2529.731488- pl=-1>
<Trade size=29 time=49- price=2529.731488- pl=1>
<Trade size=29 time=49- price=2529.731488- pl=-1>
<Trade size=29 time=49- price=2529.731488- pl=141>
<Trade size=29 time=49- price=2529.731488- pl=39>
<Trade size=29 time=49- price=2529.731488- pl=-40>
<Trade size=29 time=49- price=2529.731488- pl=104>
<Trade size=29 time=49- price=2529.731488- pl=179>
<Trade size=29 time=49- price=2529.731488- pl=195>
<Trade size=29 time=49- price=2529.731488-

AttributeError: 'Trade' object has no attribute 'side'

In [24]:
output


Start                     2024-01-17 15:15:00
End                       2024-01-21 15:00:00
Duration                      3 days 23:45:00
Exposure Time [%]                   52.083333
Equity Final [$]                 94434.829306
Equity Peak [$]                 100116.094134
Return [%]                          -5.565171
Buy & Hold Return [%]               -2.788874
Return (Ann.) [%]                  -97.953117
Volatility (Ann.) [%]                0.300014
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -5.746519
Avg. Drawdown [%]                   -3.084361
Max. Drawdown Duration        3 days 05:45:00
Avg. Drawdown Duration        1 days 21:00:00
# Trades                                   41
Win Rate [%]                        29.268293
Best Trade [%]                        0.96079
Worst Trade [%]                      -0.54063
Avg. Trade [%]                    

In [16]:
output["_trades"]

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,29,23,26,2540.895952,2527.17065,-398.033758,-0.005402,2024-01-17 21:00:00,2024-01-17 21:45:00,0 days 00:45:00
1,29,49,54,2529.731488,2532.03000,66.656848,0.000909,2024-01-18 03:30:00,2024-01-18 04:45:00,0 days 01:15:00
2,29,55,60,2530.871944,2534.60000,108.113624,0.001473,2024-01-18 05:00:00,2024-01-18 06:15:00,0 days 01:15:00
3,29,61,66,2532.072424,2535.58000,101.719704,0.001385,2024-01-18 06:30:00,2024-01-18 07:45:00,0 days 01:15:00
4,29,67,72,2540.005596,2548.20000,237.637716,0.003226,2024-01-18 08:00:00,2024-01-18 09:15:00,0 days 01:15:00
5,29,73,77,2546.348132,2532.61330,-398.310128,-0.005394,2024-01-18 09:30:00,2024-01-18 10:30:00,0 days 01:00:00
6,29,95,99,2534.743492,2521.07130,-396.493568,-0.005394,2024-01-18 15:00:00,2024-01-18 16:00:00,0 days 01:00:00
7,29,100,101,2519.127248,2505.51945,-394.626142,-0.005402,2024-01-18 16:15:00,2024-01-18 16:30:00,0 days 00:15:00
8,-30,118,118,2445.681336,2458.90335,-396.660420,-0.005406,2024-01-18 20:45:00,2024-01-18 20:45:00,0 days 00:00:00
9,-30,119,121,2445.461424,2458.68225,-396.624780,-0.005406,2024-01-18 21:00:00,2024-01-18 21:30:00,0 days 00:30:00


In [16]:
output

Start                     2024-01-17 14:30:00
End                       2024-01-21 14:15:00
Duration                      3 days 23:45:00
Exposure Time [%]                    41.40625
Equity Final [$]                 97540.615074
Equity Peak [$]                  100578.45718
Return [%]                          -2.459385
Buy & Hold Return [%]               -3.285163
Return (Ann.) [%]                  -83.761693
Volatility (Ann.) [%]                0.968653
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -3.044302
Avg. Drawdown [%]                   -1.839568
Max. Drawdown Duration        1 days 21:45:00
Avg. Drawdown Duration        1 days 08:00:00
# Trades                                   32
Win Rate [%]                            31.25
Best Trade [%]                        0.96079
Worst Trade [%]                      -0.54063
Avg. Trade [%]                    

In [17]:
output["_trades"]["ReturnPct"]

0    -0.003170
1    -0.001704
2     0.004441
3    -0.005402
4    -0.002795
5     0.006864
6     0.009608
7    -0.005402
8    -0.005406
9    -0.005402
10   -0.005406
11   -0.005402
12    0.000493
13   -0.002716
14   -0.002755
15   -0.001643
16    0.001304
17    0.003666
18   -0.000773
19   -0.001125
20   -0.001669
21   -0.000497
22    0.003026
23    0.000377
24   -0.002959
25   -0.002125
26   -0.001511
27   -0.002112
28   -0.001122
29    0.000330
30   -0.002864
31    0.000076
Name: ReturnPct, dtype: float64

In [18]:
bt_traffic_light.plot()


Row(id='1470', ...)

In [ ]:
global_trades = []
class ImprovedTrafficLight2(Strategy):
    ema_period_red = 127
    ema_period_yellow = 22
    ema_period_green = 6
    rsi_period = 24
    upper_band_multiplier = 2
    sl_multiplier = 0.99
    tp_multiplier = 1.02
    tp_for_short =  0.98
    sl_for_short = 1.01
    depth_params_2 = 1
    size = 0.15
    lookback_period = 15
    max_open_positions = 2

    def init(self):
        self.open_positions = 0
        self.trade_log = []
        self.close = self.data.Close
        self.high = self.data.High
        self.low = self.data.Low
        self.volume = self.data.Volume
        self.green = self.I(_get_ema, self.data.Close, self.data.index, self.ema_period_green, name = "ema_6", color = 'green')
        self.yellow = self.I(_get_ema, self.data.Close, self.data.index, self.ema_period_yellow, name = "ema_22", color = 'yellow')
        self.red = self.I(_get_ema, self.data.Close, self.data.index, self.ema_period_red, name = "ema_127", color = 'red')
        self.volume_ma = self.I(_get_sma, self.data.Volume, 20)
        self.rsi = self.I(_get_rsi, self.data.Close, self.data.index, self.rsi_period)
        self.macd = self.I(_get_macd_main_line, self.data.Close, self.data.index, 12, 26, 9)
        self.bollinger_upper, self.bollinger_lower = self.I(_get_bollinger_bands, self.data.Close, self.data.index, 20, 2)
        self.stochastic_k, self.stochastic_d = self.I(_get_stochastic_oscillator, self.data.High, self.data.Low, self.data.Close, self.data.index, 14, 3)
        self.atr = self.I(_get_atr, self.data.High, self.data.Low, self.data.Close, self.data.index, 14)

    def log_trade(self, trade_action, trade_data):
        global global_trades
        # Отслеживаем начало и конец сделки
        if trade_action == 'entry':
            trade_data['entry_price'] = trade_data['Price']
        elif trade_action == 'exit':
            trade_data['exit_price'] = trade_data['Price']
        global_trades.append(trade_data)

    def adjust_tp_sl(self, close_price, trend_strength):
        """
        Адаптируем SL и TP в зависимости от силы тренда.
        trend_strength: значение, отражающее силу тренда (например, разница между EMA).
        """
        # при сильном тренде увеличиваем TP и уменьшаем SL
        if trend_strength > 30:
            tp_multiplier = 1.10  # Увеличиваем TP
            sl_multiplier = 0.95  # Уменьшаем SL
        else:
            tp_multiplier = self.tp_multiplier  # Используем стандартные значения
            sl_multiplier = self.sl_multiplier

        return close_price * sl_multiplier, close_price * tp_multiplier

    def next(self):
        # if self.open_positions >= self.max_open_positions:
        #     return
        current_time = self.data.index[-1]
        current_volume = self.volume[-1]
        close_price, high, low = self.data.Close[-1], self.data.High[-1], self.data.Low[-1]
        red, yellow, green = self.red[-1], self.yellow[-1], self.green[-1]
        prev_red, prev_yellow, prev_green = self.red[-2], self.yellow[-2], self.green[-2]
        prev_close, prev_high, prev_low = self.data.Close[-2], self.data.High[-2], self.data.Low[-2]
        upper_band, lower_band = self.bollinger_upper[-1], self.bollinger_lower[-1]
        rsi_current = self.rsi[-1]
        bullish_divergence, bearish_divergence = _get_rsi_divergence(self.data.Close, self.rsi, self.lookback_period)

        trend_strength = abs(self.green[-1] - self.red[-1])  # вычисление силы тренда
        if self.should_short(red, yellow, green,
                            prev_red, prev_yellow, prev_green,
                            high, prev_high, close_price, prev_close, current_volume,
                            rsi_current, upper_band, bearish_divergence):
            sl, tp = self.adjust_tp_sl(close_price, trend_strength)
            self.sell(size=self.size, sl=tp, tp=sl-0.05)
            self.log_trade('entry', {'Time': current_time, 'Price': close_price, 'Type': 'Short'})
            # if self.position:
            #     self.open_positions += 1
        elif self.should_long(red, yellow, green,
                              prev_red, prev_yellow, prev_green,
                              low, prev_low, close_price, prev_close, current_volume,
                              rsi_current, upper_band, bullish_divergence):
            sl, tp = self.adjust_tp_sl(close_price, trend_strength)
            self.buy(size=self.size, sl=sl, tp=tp)
            self.log_trade('entry', {'Time': current_time, 'Price': close_price, 'Type': 'Long'})
            # if self.position:
            #     self.open_positions += 1

    def should_short(self, red, yellow, green,
                     prev_red, prev_yellow, prev_green,
                     high, prev_high, close_price, prev_close, current_volume,
                     rsi_current, upper_band, bearish_divergence):
        price_above_upper_band = high >= upper_band
        overbought_rsi = rsi_current < 45
        return red >= yellow + self.depth_params_2 and yellow >= green + self.depth_params_2 and overbought_rsi and not bearish_divergence and price_above_upper_band

    def should_long(self, red, yellow, green,
                    prev_red, prev_yellow, prev_green,
                    low, prev_low, close_price, prev_close, current_volume,
                    rsi_current, lower_band, bullish_divergence):
        price_below_lower_band = low <= lower_band
        oversold_rsi = rsi_current > 70
        return red <= yellow - self.depth_params_2 and yellow <= green - self.depth_params_2 and oversold_rsi and not bullish_divergence

    # def on_trade(self, trade):
    #     # Этот метод вызывается при каждой сделке автоматически фреймворком
    #     if trade.is_closed:
    #         # Уменьшаем счетчик при закрытии сделки
    #         self.open_positions = max(0, self.open_positions - 1)

bt_traffic_light = Backtest(data, ImprovedTrafficLight2, cash=10000, commission=commission, margin=0.1)
output = bt_traffic_light.run()

In [ ]:
fig = px.line(output['_equity_curve'], y='Equity', title='Equity Over Time')
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Equity')
fig.show()

fig = px.line(data, y='Open', title='Open Price Over Time')
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Open')
fig.show()